In [48]:
import pandas as pd

Now import the dataset

In [49]:
dataset = pd.read_csv('wfp_food_price_bgd.csv')
dataset

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
0,7/15/1998,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182
1,7/15/1998,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101
2,8/15/1998,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620
3,8/15/1998,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053
4,9/15/1998,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16519,8/15/2024,Sylhet,Sunamganj,Sunamgonj Sadar,25.072282,91.400318,pulses and nuts,Lentils (masur),KG,aggregate,Retail,BDT,122.50,1.0390
16520,8/15/2024,Sylhet,Sylhet,Sylhet Sadar,24.890531,91.871936,cereals and tubers,Rice (coarse),KG,aggregate,Retail,BDT,51.50,0.4368
16521,8/15/2024,Sylhet,Sylhet,Sylhet Sadar,24.890531,91.871936,cereals and tubers,Wheat flour,KG,aggregate,Retail,BDT,60.00,0.5089
16522,8/15/2024,Sylhet,Sylhet,Sylhet Sadar,24.890531,91.871936,oil and fats,Oil (palm),L,aggregate,Retail,BDT,137.50,1.1662


In [50]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16524 entries, 0 to 16523
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       16524 non-null  object 
 1   admin1     16524 non-null  object 
 2   admin2     16524 non-null  object 
 3   market     16524 non-null  object 
 4   latitude   16524 non-null  float64
 5   longitude  16524 non-null  float64
 6   category   16524 non-null  object 
 7   commodity  16524 non-null  object 
 8   unit       16524 non-null  object 
 9   priceflag  16524 non-null  object 
 10  pricetype  16524 non-null  object 
 11  currency   16524 non-null  object 
 12  price      16524 non-null  float64
 13  usdprice   16524 non-null  float64
dtypes: float64(4), object(10)
memory usage: 1.8+ MB


In [51]:
dataset = dataset.dropna()  # drop rows with missing values

Now convert the date column into useful features (date)

In [52]:
dataset['year'] = pd.to_datetime(dataset['date']).dt.year
dataset['month'] = pd.to_datetime(dataset['date']).dt.month
dataset['day'] = pd.to_datetime(dataset['date']).dt.day

In [53]:
dataset.head()

,date,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day
0,7/15/1998,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15
1,7/15/1998,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15
2,8/15/1998,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15
3,8/15/1998,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15
4,9/15/1998,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15


In [54]:
dataset = dataset.drop('date', axis=1)  
dataset.head()

,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day
0,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15
1,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15
2,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15
3,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15
4,Dhaka,Dhaka,Dhaka,23.81,90.41,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15


In [56]:
dataset['market'].unique()

array(['Dhaka', 'Dhaka Division', 'Chittagong Division',
       'Khulna Division', 'Rajshahi Division', 'Barisal Division',
       'Sylhet Division', 'Rangpur Sadar', 'Barguna Sadar',
       'Barisal Sadar', 'Bhola Sadar', 'Jhalokati Sadar',
       'Patuakhali Sadar', 'Pirojpur Sadar', 'Bandarban Sadar',
       'Brahmonbaria Sadar', 'Chandpur Sadar', 'Chittagong Sadar',
       'Patiya Sadar', 'Comilla Sadar', 'Cox`s Bazar Sadar', 'Feni Sadar',
       'Khagrachari Sadar', 'Laxmipur Sadar', 'Noakhali Sadar',
       'Rangamati Sadar', 'Dhaka Sadar', 'Faridpur Sadar',
       'Gazipur Sadar', 'Gopalgonj Sadar', 'Kishoregonj Sadar',
       'Madaripur Sadar', 'Manikganj Sadar', 'Munshiganj Sadar',
       'Narayanganj Sadar', 'Narsingdi Sadar', 'Rajbari Sadar',
       'Shariatpur Sadar', 'Tangail sadar Market', 'Bagerhat Sadar',
       'Chuadanga Sadar', 'Jessore Sadar', 'Jhenaidah Sadar',
       'Khulna Sadar', 'Kushtia Sadar', 'Magura Sadar', 'Meherpur Sadar',
       'Narail Sadar', 'Satkhir

In [57]:
from fuzzywuzzy import process

In [58]:
markets = dataset['market'].str.lower()

In [59]:
reference = ['Dhaka', 'Chittagong', 'Khulna', 'Rajshahi', 'Barisal', 'Sylhet', 'Rangpur', 'Mymensingh', 
             'Barguna', 'Bhola ', 'Jhalokati ',
       'Patuakhali ', 'Pirojpur ', 'Bandarban ',
       'Brahmonbaria ', 'Chandpur ', 
       'Patiya ', 'Comilla ', 'CoxsBazar ', 'Feni ',
       'Khagrachari ', 'Laxmipur ', 'Noakhali ',
       'Rangamati ', 'Faridpur ',
       'Gazipur ', 'Gopalgonj ', 'Kishoregonj ',
       'Madaripur ', 'Manikganj ', 'Munshiganj ',
       'Narayanganj ', 'Narsingdi ', 'Rajbari ',
       'Shariatpur ', 'Tangail  ', 'Bagerhat ',
       'Chuadanga ', 'Jessore ', 'Jhenaidah ',
       'Kushtia ', 'Magura ', 'Meherpur ',
       'Narail ', 'Satkhira ', 'Jamalpur ',
       'Netrakona ', 'Sherpur ',
       'Bogra ', 'Jaipurhat ', 'Naogaon ', 'Natore ',
       'Nawabgonj ', 'Pabna ',
       'Sirajgonj ', 'Dinajpur ', 'Gaibandha ',
       'Kurigram ', 'Lalmonirhat ', 'Nilphamari ',
       'Panchagarh ',
       'Thakurgaon ', 'Habigonj ', 'Maulavibazar ',
       'Sunamgonj ', 'BhashanChar', 'Teknaf ',
       'Ukhia ']

In [60]:
grouped_markets = {market: process.extractOne(market, reference)[0] for market in markets}

print(grouped_markets)

{'dhaka': 'Dhaka', 'dhaka division': 'Dhaka', 'chittagong division': 'Chittagong', 'khulna division': 'Khulna', 'rajshahi division': 'Rajshahi', 'barisal division': 'Barisal', 'sylhet division': 'Sylhet', 'rangpur sadar': 'Rangpur', 'barguna sadar': 'Barguna', 'barisal sadar': 'Barisal', 'bhola sadar': 'Bhola ', 'jhalokati sadar': 'Jhalokati ', 'patuakhali sadar': 'Patuakhali ', 'pirojpur sadar': 'Pirojpur ', 'bandarban sadar': 'Bandarban ', 'brahmonbaria sadar': 'Brahmonbaria ', 'chandpur sadar': 'Chandpur ', 'chittagong sadar': 'Chittagong', 'patiya sadar': 'Patiya ', 'comilla sadar': 'Comilla ', 'cox`s bazar sadar': 'CoxsBazar ', 'feni sadar': 'Feni ', 'khagrachari sadar': 'Khagrachari ', 'laxmipur sadar': 'Laxmipur ', 'noakhali sadar': 'Noakhali ', 'rangamati sadar': 'Rangamati ', 'dhaka sadar': 'Dhaka', 'faridpur sadar': 'Faridpur ', 'gazipur sadar': 'Gazipur ', 'gopalgonj sadar': 'Gopalgonj ', 'kishoregonj sadar': 'Kishoregonj ', 'madaripur sadar': 'Madaripur ', 'manikganj sadar'

In [74]:
dataset_subset = pd.DataFrame(dataset, columns=['market', 'latitude', 'longitude'])

In [75]:
dataset_subset['market'] = dataset_subset['market'].replace(grouped_markets)

In [76]:
cleaned_data = dataset_subset.groupby('market', as_index=False).agg({
    'latitude': 'mean',
    'longitude': 'mean'
})

In [77]:
print(cleaned_data)

                  market   latitude  longitude
0         Bagerhat Sadar  22.665347  89.792432
1        Bandarban Sadar  22.219697  92.221897
2          Barguna Sadar  22.161255  90.120285
3       Barisal Division  22.701944  90.371111
4          Barisal Sadar  22.712250  90.360876
..                   ...        ...        ...
71          Sylhet Sadar  24.890531  91.871936
72  Tangail sadar Market  24.250265  89.913873
73         Teknaf Market  20.870600  92.298115
74      Thakurgaon Sadar  26.020304  88.469250
75          Ukhia Market  21.242928  92.140437

[76 rows x 3 columns]


In [78]:
dataset

,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day
0,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15
1,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15
2,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15
3,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15
4,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16519,Sylhet,Sunamganj,Sunamgonj Sadar,25.072282,91.400318,pulses and nuts,Lentils (masur),KG,aggregate,Retail,BDT,122.50,1.0390,2024,8,15
16520,Sylhet,Sylhet,Sylhet Sadar,24.890531,91.871936,cereals and tubers,Rice (coarse),KG,aggregate,Retail,BDT,51.50,0.4368,2024,8,15
16521,Sylhet,Sylhet,Sylhet Sadar,24.890531,91.871936,cereals and tubers,Wheat flour,KG,aggregate,Retail,BDT,60.00,0.5089,2024,8,15
16522,Sylhet,Sylhet,Sylhet Sadar,24.890531,91.871936,oil and fats,Oil (palm),L,aggregate,Retail,BDT,137.50,1.1662,2024,8,15


In [79]:
dataset['market'] = dataset['market'].replace(grouped_markets)

In [81]:
dataset['market'].unique()

array(['Dhaka', 'Dhaka Division', 'Chittagong Division',
       'Khulna Division', 'Rajshahi Division', 'Barisal Division',
       'Sylhet Division', 'Rangpur Sadar', 'Barguna Sadar',
       'Barisal Sadar', 'Bhola Sadar', 'Jhalokati Sadar',
       'Patuakhali Sadar', 'Pirojpur Sadar', 'Bandarban Sadar',
       'Brahmonbaria Sadar', 'Chandpur Sadar', 'Chittagong Sadar',
       'Patiya Sadar', 'Comilla Sadar', 'Cox`s Bazar Sadar', 'Feni Sadar',
       'Khagrachari Sadar', 'Laxmipur Sadar', 'Noakhali Sadar',
       'Rangamati Sadar', 'Dhaka Sadar', 'Faridpur Sadar',
       'Gazipur Sadar', 'Gopalgonj Sadar', 'Kishoregonj Sadar',
       'Madaripur Sadar', 'Manikganj Sadar', 'Munshiganj Sadar',
       'Narayanganj Sadar', 'Narsingdi Sadar', 'Rajbari Sadar',
       'Shariatpur Sadar', 'Tangail sadar Market', 'Bagerhat Sadar',
       'Chuadanga Sadar', 'Jessore Sadar', 'Jhenaidah Sadar',
       'Khulna Sadar', 'Kushtia Sadar', 'Magura Sadar', 'Meherpur Sadar',
       'Narail Sadar', 'Satkhir

In [88]:
dataset['market'] = dataset['market'].str.replace('Sadar', '', regex=False)

In [89]:
print(dataset['market'].unique())

['Dhaka' 'Dhaka Division' 'Chittagong Division' 'Khulna Division'
 'Rajshahi Division' 'Barisal Division' 'Sylhet Division' 'Rangpur '
 'Barguna ' 'Barisal ' 'Bhola ' 'Jhalokati ' 'Patuakhali ' 'Pirojpur '
 'Bandarban ' 'Brahmonbaria ' 'Chandpur ' 'Chittagong ' 'Patiya '
 'Comilla ' 'Cox`s Bazar ' 'Feni ' 'Khagrachari ' 'Laxmipur ' 'Noakhali '
 'Rangamati ' 'Dhaka ' 'Faridpur ' 'Gazipur ' 'Gopalgonj ' 'Kishoregonj '
 'Madaripur ' 'Manikganj ' 'Munshiganj ' 'Narayanganj ' 'Narsingdi '
 'Rajbari ' 'Shariatpur ' 'Tangail sadar Market' 'Bagerhat ' 'Chuadanga '
 'Jessore ' 'Jhenaidah ' 'Khulna ' 'Kushtia ' 'Magura ' 'Meherpur '
 'Narail ' 'Satkhira ' 'Jamalpur ' 'Mymensing ' 'Netrakona ' 'Sherpur '
 'Bogra ' 'Jaipurhat ' 'Naogaon ' 'Natore ' 'Nawabgonj ' 'Pabna '
 'Rajshahi ' 'Sirajgonj ' 'Dinajpur ' 'Gaibandha ' 'Kurigram '
 'Lalmonirhat ' 'Nilphamari ' 'Panchagarh ' 'Rangpur Municipality Market'
 'Thakurgaon ' 'Habigonj ' 'Maulavibazar ' 'Sunamgonj ' 'Sylhet '
 'Bhashan Char' 'Teknaf Mark

In [90]:
dataset['market'] = dataset['market'].str.replace('Division', '', regex=False)

In [91]:
dataset['market'] = dataset['market'].str.replace('Market', '', regex=False)

In [92]:
dataset['market'] = dataset['market'].str.replace(' ', '', regex=False)

In [93]:
print(dataset['market'].unique())

['Dhaka' 'Chittagong' 'Khulna' 'Rajshahi' 'Barisal' 'Sylhet' 'Rangpur'
 'Barguna' 'Bhola' 'Jhalokati' 'Patuakhali' 'Pirojpur' 'Bandarban'
 'Brahmonbaria' 'Chandpur' 'Patiya' 'Comilla' 'Cox`sBazar' 'Feni'
 'Khagrachari' 'Laxmipur' 'Noakhali' 'Rangamati' 'Faridpur' 'Gazipur'
 'Gopalgonj' 'Kishoregonj' 'Madaripur' 'Manikganj' 'Munshiganj'
 'Narayanganj' 'Narsingdi' 'Rajbari' 'Shariatpur' 'Tangailsadar'
 'Bagerhat' 'Chuadanga' 'Jessore' 'Jhenaidah' 'Kushtia' 'Magura'
 'Meherpur' 'Narail' 'Satkhira' 'Jamalpur' 'Mymensing' 'Netrakona'
 'Sherpur' 'Bogra' 'Jaipurhat' 'Naogaon' 'Natore' 'Nawabgonj' 'Pabna'
 'Sirajgonj' 'Dinajpur' 'Gaibandha' 'Kurigram' 'Lalmonirhat' 'Nilphamari'
 'Panchagarh' 'RangpurMunicipality' 'Thakurgaon' 'Habigonj' 'Maulavibazar'
 'Sunamgonj' 'BhashanChar' 'Teknaf' 'Ukhia']


In [94]:
dataset

,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day
0,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15
1,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15
2,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15
3,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15
4,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16519,Sylhet,Sunamganj,Sunamgonj,25.072282,91.400318,pulses and nuts,Lentils (masur),KG,aggregate,Retail,BDT,122.50,1.0390,2024,8,15
16520,Sylhet,Sylhet,Sylhet,24.890531,91.871936,cereals and tubers,Rice (coarse),KG,aggregate,Retail,BDT,51.50,0.4368,2024,8,15
16521,Sylhet,Sylhet,Sylhet,24.890531,91.871936,cereals and tubers,Wheat flour,KG,aggregate,Retail,BDT,60.00,0.5089,2024,8,15
16522,Sylhet,Sylhet,Sylhet,24.890531,91.871936,oil and fats,Oil (palm),L,aggregate,Retail,BDT,137.50,1.1662,2024,8,15


In [95]:
from geopy.distance import geodesic

In [96]:
reference_point = (23.8103, 90.4125)

In [97]:
cleaned_data['distance_from_center'] = cleaned_data.apply(
    lambda row: geodesic((row['latitude'], row['longitude']), reference_point).kilometers, axis=1
)

In [98]:
print(cleaned_data)

                  market   latitude  longitude  distance_from_center
0         Bagerhat Sadar  22.665347  89.792432            141.792842
1        Bandarban Sadar  22.219697  92.221897            255.791244
2          Barguna Sadar  22.161255  90.120285            185.062870
3       Barisal Division  22.701944  90.371111            122.820839
4          Barisal Sadar  22.712250  90.360876            121.721182
..                   ...        ...        ...                   ...
71          Sylhet Sadar  24.890531  91.871936            190.386704
72  Tangail sadar Market  24.250265  89.913873             70.338741
73         Teknaf Market  20.870600  92.298115            379.059884
74      Thakurgaon Sadar  26.020304  88.469250            313.779047
75          Ukhia Market  21.242928  92.140437            335.292136

[76 rows x 4 columns]


In [102]:
dataset

,admin1,admin2,market,latitude_x,longitude_x,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day,latitude_y,longitude_y,distance_from_center
0,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15,23.81,90.41,0.256909
1,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15,23.81,90.41,0.256909
2,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15,23.81,90.41,0.256909
3,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15,23.81,90.41,0.256909
4,Dhaka,Dhaka,Dhaka,23.810000,90.410000,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15,23.81,90.41,0.256909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2189,Dhaka,Dhaka,Dhaka,23.733557,90.383758,pulses and nuts,Lentils (masur),KG,aggregate,Retail,BDT,107.50,0.9150,2024,7,15,23.81,90.41,0.256909
2190,Dhaka,Dhaka,Dhaka,23.733557,90.383758,cereals and tubers,Rice (coarse),KG,aggregate,Retail,BDT,54.00,0.4580,2024,8,15,23.81,90.41,0.256909
2191,Dhaka,Dhaka,Dhaka,23.733557,90.383758,cereals and tubers,Wheat flour,KG,aggregate,Retail,BDT,57.50,0.4877,2024,8,15,23.81,90.41,0.256909
2192,Dhaka,Dhaka,Dhaka,23.733557,90.383758,oil and fats,Oil (palm),L,aggregate,Retail,BDT,127.50,1.0814,2024,8,15,23.81,90.41,0.256909


In [104]:
# Combine latitude and longitude columns
dataset['latitude'] = dataset['latitude_y'].fillna(dataset['latitude_x'])
dataset['longitude'] = dataset['longitude_y'].fillna(dataset['longitude_x'])


In [105]:
# Drop the old columns
dataset = dataset.drop(columns=['latitude_x', 'latitude_y', 'longitude_x', 'longitude_y'])

In [106]:
dataset.head()

,admin1,admin2,market,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day,distance_from_center,latitude,longitude
0,Dhaka,Dhaka,Dhaka,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15,0.256909,23.81,90.41
1,Dhaka,Dhaka,Dhaka,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15,0.256909,23.81,90.41
2,Dhaka,Dhaka,Dhaka,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15,0.256909,23.81,90.41
3,Dhaka,Dhaka,Dhaka,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15,0.256909,23.81,90.41
4,Dhaka,Dhaka,Dhaka,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15,0.256909,23.81,90.41


In [109]:
dataset = dataset.drop(columns=['admin1', 'admin2'])

In [125]:
dataset.head()

,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day,distance_from_center,latitude,longitude,market_Dhaka,market_0
0,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15,0.256909,23.81,90.41,True,True
1,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15,0.256909,23.81,90.41,True,True
2,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15,0.256909,23.81,90.41,True,True
3,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15,0.256909,23.81,90.41,True,True
4,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15,0.256909,23.81,90.41,True,True


In [122]:
from sklearn.preprocessing import LabelEncoder

In [123]:
le = LabelEncoder()

In [124]:
dataset['market_encoded'] = le.fit_transform(dataset['market'])
dataset

KeyError: 'market'

In [120]:
dataset = pd.get_dummies(dataset, columns=['market'], prefix='market')

KeyError: "None of [Index(['market'], dtype='object')] are in the [columns]"

In [119]:
dataset.head()

,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day,distance_from_center,latitude,longitude,market_Dhaka,market_0
0,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15,0.256909,23.81,90.41,True,True
1,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15,0.256909,23.81,90.41,True,True
2,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15,0.256909,23.81,90.41,True,True
3,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15,0.256909,23.81,90.41,True,True
4,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15,0.256909,23.81,90.41,True,True


In [117]:
dataset

,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,day,distance_from_center,latitude,longitude,market_encoded,market_Dhaka
0,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1261.40,27.0182,1998,7,15,0.256909,23.81,90.41,0,True
1,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1055.60,22.6101,1998,7,15,0.256909,23.81,90.41,0,True
2,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1288.75,27.3620,1998,8,15,0.256909,23.81,90.41,0,True
3,cereals and tubers,Wheat,100 KG,actual,Wholesale,BDT,1060.00,22.5053,1998,8,15,0.256909,23.81,90.41,0,True
4,cereals and tubers,"Rice (coarse, BR-8/ 11/, Guti Sharna)",100 KG,actual,Wholesale,BDT,1415.50,30.0531,1998,9,15,0.256909,23.81,90.41,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2189,pulses and nuts,Lentils (masur),KG,aggregate,Retail,BDT,107.50,0.9150,2024,7,15,0.256909,23.81,90.41,0,True
2190,cereals and tubers,Rice (coarse),KG,aggregate,Retail,BDT,54.00,0.4580,2024,8,15,0.256909,23.81,90.41,0,True
2191,cereals and tubers,Wheat flour,KG,aggregate,Retail,BDT,57.50,0.4877,2024,8,15,0.256909,23.81,90.41,0,True
2192,oil and fats,Oil (palm),L,aggregate,Retail,BDT,127.50,1.0814,2024,8,15,0.256909,23.81,90.41,0,True
